In [11]:
import tempfile
import sys
import os
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

In [ ]:
# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,
    # python_executable=sys.executable # to use the current environment
)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="NEVER",  # Always take human input for this agent for safety.
)

In [13]:
temp_dir.name

'C:\\Users\\anoop\\AppData\\Local\\Temp\\tmp8e2pferp'

In [14]:
message_with_code_block = """This is a message with code block.
The code block is below:
```python
import sys
import os
print("Python exec:", sys.executable)
import numpy as np
os.environ["MPLBACKEND"] = "agg"
import matplotlib.pyplot as plt
x = np.random.randint(0, 100, 100)
y = np.random.randint(0, 100, 100)
plt.scatter(x, y)
plt.savefig('scatter.png')
print('Scatter plot saved to scatter.png')
```
This is the end of the message.
"""

In [16]:
print(code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}]))


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Python exec: C:\Users\anoop\AppData\Local\Programs\Python\Python311\python.exe
Scatter plot saved to scatter.png



In [27]:
import tempfile
import sys
import os
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

sys.path.append(os.path.abspath(".."))
from llm_config import llm_config

temp_dir = tempfile.TemporaryDirectory()
executor = LocalCommandLineCodeExecutor(timeout=10, work_dir=temp_dir.name)

code_generator_agent = ConversableAgent(
    "code_generator_agent",
    llm_config=llm_config,
    system_message="You are a helpful assistant that writes executable Python code."
                   "Write a Python function based on user request."
                   "Ensure to provide `os.environ['MPLBACKEND'] = 'agg'` at the beginning of the code before any imports."
                   "The code should be in a markdown code block using ```python ... ```.",
)

code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=llm_config,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER",
)

generation_reply = code_generator_agent.generate_reply(
    messages=[{"role": "user", "content": "Write Python code that generate random values, "
               "plot those values in a matplotlib histogram and save the plot as an image in the temporary directory."
               "The data should also be saved in csv format at the same location."
               "print the path to access the files saved."}]
)

print("Generated Code:\n", generation_reply)

execution_reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": generation_reply}])
print("\nExecution Output:\n", execution_reply)


>>>>>>>> USING AUTO REPLY...
Generated Code:
 ```python
import os
os.environ['MPLBACKEND'] = 'agg'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tempfile

def generate_plot_and_save():
    # Generate random data
    data = np.random.randn(1000)
    
    # Create a histogram
    plt.hist(data, bins=30, alpha=0.7, color='blue')
    plt.title('Histogram of Random Values')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    
    # Save the plot in a temporary directory
    temp_dir = tempfile.gettempdir()
    plot_path = os.path.join(temp_dir, 'histogram.png')
    plt.savefig(plot_path)
    plt.close()
    
    # Save the data in CSV format
    data_path = os.path.join(temp_dir, 'data.csv')
    df = pd.DataFrame(data, columns=['Random Values'])
    df.to_csv(data_path, index=False)
    
    # Print the paths
    print(f'Plot saved at: {plot_path}')
    print(f'Data saved at: {data_path}')

# Execute the function
generate_plot_and_save()
```

>>>>>>>> EX

In [39]:
import tempfile
import sys
import os
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor
import subprocess

sys.path.append(os.path.abspath(".."))
from llm_config import llm_config

temp_dir = tempfile.TemporaryDirectory()
executor = LocalCommandLineCodeExecutor(timeout=20, work_dir=temp_dir.name)

code_generator_agent = ConversableAgent(
    "code_generator_agent",
    llm_config=llm_config,
    system_message=(
        "You are a helpful AI that writes executable Python code. "
        "Always output Python code in markdown code blocks (```python ... ```)." \
        "Don't provide any help text like the libraries required for the code and provide only the python code blocks."
    ),
)

code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=llm_config,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER",
    system_message=(
        "You execute the provided Python code and return the execution result, "
        "including any errors or tracebacks."
    ),
)

code_reviewer_agent = ConversableAgent(
    "code_reviewer_agent",
    llm_config=llm_config,
    system_message=(
        "You are a diagnostic agent that reviews the output of executed Python code. "
        "If there are missing module errors (like 'ModuleNotFoundError'), identify the missing package "
        "and respond with the list of packages separated with commas so that it can be used for installations directly using pip."
        "If no error or missing package, reply with 'No action needed'."
    ),
)

generation_reply = code_generator_agent.generate_reply(
    messages=[{"role": "user", "content": "Write Python code that generate random numbers and display in pandas dataframe."}])
print("Generated Code:\n", generation_reply)

execution_reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": generation_reply}])
print("\nExecution Output:\n", execution_reply)

review_reply = code_reviewer_agent.generate_reply(messages=[{"role": "user", "content": execution_reply}])
print("\nReview Output:\n", review_reply)

if "No action needed" not in review_reply:
    print("\nInstalling Missing Library...")
    subprocess.run(["python", "-m", "pip", "install", review_reply], check=True)
    print("\nRetrying Code Execution...")
    retry_reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": generation_reply}])
    print("\nFinal Execution Output:\n", retry_reply)
else:
    print("\nNo installation required.")


>>>>>>>> USING AUTO REPLY...
Generated Code:
 ```python
import pandas as pd
import numpy as np

# Generate random numbers
random_numbers = np.random.random(size=(10, 5))  # 10 rows and 5 columns

# Create a DataFrame
df = pd.DataFrame(random_numbers, columns=[f'Col_{i+1}' for i in range(5)])

# Display the DataFrame
print(df)
```

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...

Execution Output:
 exitcode: 1 (execution failed)
Code output: Traceback (most recent call last):
  File "C:\Users\anoop\AppData\Local\Temp\tmp2dq0llu4\tmp_code_aa517277f82f2ee4fb235f014429a580.py", line 1, in <module>
    import pandas as pd
ModuleNotFoundError: No module named 'pandas'


>>>>>>>> USING AUTO REPLY...

Review Output:
 pandas

Installing Missing Library...

Retrying Code Execution...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...

Final Execution Output:
 exitcode: 0 (execution succeeded)
Code output:       Col_1     Col_2     Col_3     Col_4     Col_5
0  0.39899